In [17]:
import json
import logging
import os
import sys

from geopy.geocoders import Bing

from dotenv import load_dotenv
from loguru import logger
import numpy as np
import pandas as pd
import s3fs
import sqlalchemy
from sqlalchemy import text

import geopandas as gpd
from shapely.geometry import LineString

sys.path.append('..')

from src.preprocessing import data_preprocessing_bridge

pd.set_option('display.max_columns', None)

%load_ext autoreload
%autoreload 2

project_path = 's3://sfgdata/projects/sustainable-transport'
dataset_name = 'sql_poc4'
dict_column_names = 'bridge_names'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
df = pd.read_csv(f'{project_path}/raw/{dataset_name}.csv')

In [9]:
# Dict for column names
s3 = s3fs.S3FileSystem()
with s3.open(f'{project_path}/external/{dict_column_names}.json', 'r') as fp:
    dict_names = json.load(fp)
dict_names['Shipment id'] = 'TKNUM'
dict_names['IG'] = 'IG'

logger.info('Import done')

# Cleaning df_raw
df = data_preprocessing_bridge(df, dict_names)

2023-03-27 10:52:29.584 | INFO     | __main__:<module>:8 - Import done
c:\Users\SESA667490\Documents\GitHub\sustainable-transport\notebooks\..\src\preprocessing.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[i] = df[i].str.upper()
c:\Users\SESA667490\Documents\GitHub\sustainable-transport\notebooks\..\src\preprocessing.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[i] = df[i].apply(lambda x: clean_string(x))
c:\Users\SESA667490\Documents\GitHub\sustainable-transport\notebooks\..\src\prepro

## From country codes to countries

In [10]:
from src.preprocessing import get_country_name
get_country_name(df)

,Pickup date,DC code,DC name,Client no,Client name,Address,Zip,City,Country code,Shipping type,Carrier name,Sender weight (kg),Volume (m3),Nb of packages,Shipment id,IG,Country
0,2022-01-04 07:44:00,FR45,DC Evreux,1000072718,ENEDIS,46/48 BD DEQUEVAUVILLIERS,92230,GENNEVILLIERS,FR,49,TRANS SERVICE,117.000,0.720,NaN,CV01208211,0,France
1,2022-01-10 11:19:00,FR43,DC Newlog,1000053993,SALENTEY AGENCE BEAUVAIS,1 RUE DU WAGE,60000,BEAUVAIS,FR,49,KUEHNE & NAGEL ROAD,138.360,6.768,NaN,CS01167139,0,France
2,2022-01-05 18:05:00,HU07,HU08-TPP RDC Cent.,1500013142,23,12,1612,,BG,49,Gebrüder Weiss Szállítmányozási,6.700,0.108,NaN,0001262428,0,Bulgaria
3,2022-01-05 10:59:00,FR45,DC Evreux,1000064589,POWER PANELS SA,12 RUE DE BITBOURG,1273,LUXEMBOURG,LU,49,SCHENKER FRANCE,285.002,2.377,NaN,CV01212761,1,Luxembourg
4,2022-01-03 11:16:00,FR43,DC Newlog,1000058890,ITEC PRODUCTION,10 AVENUE DES CARREAUX ZI LES FOU,49480,VERRIERES EN ANJOU,FR,49,KUEHNE & NAGEL ROAD,50.948,3.438,NaN,CS01162594,0,France
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
634116,2022-10-07 16:52:00,HU07,HU08-TPP RDC Cent.,VFR02531,SCHNEIDER ELECTRIC INDUSTRIES SAS,RUE DES GARINNES,38070,SAINT QUENTIN FALLAVIER,FR,42,Gebrüder Weiss Szállítmányozási,271.000,1.728,NaN,0001360655,0,France
634117,2022-08-11 17:48:00,HU07,HU08-TPP RDC Cent.,VFR02531,SCHNEIDER ELECTRIC INDUSTRIES SAS,RUE DES GARINNES,38070,SAINT QUENTIN FALLAVIER,FR,42,Gebrüder Weiss Szállítmányozási,1586.769,19.749,NaN,0001340692,0,France
634118,2022-09-14 17:33:00,HU07,HU08-TPP RDC Cent.,VFR08021,SCHNEIDER ELECTRIC INDUSTRIES SAS,RUE ROLLAND GARROS,27930,GUICHAINVILLE,FR,42,Gebrüder Weiss Szállítmányozási,5.000,0.108,NaN,0001352109,0,France
634119,2022-12-14 11:23:00,HU07,HU08-TPP RDC Cent.,VFR08021,SCHNEIDER ELECTRIC INDUSTRIES SAS,RUE ROLLAND GARROS,27930,GUICHAINVILLE,FR,42,Gebrüder Weiss Szállítmányozási,62.010,0.634,NaN,0001383910,0,France


* for better results integrating country code

## Get geolocation

In [11]:
df = pd.read_csv(f'{project_path}/processed/{dataset_name}_routes.csv')

In [14]:
df

,DC code,Client no,Address,City,Zip,Country,Country code
0,FR45,1000072718,46/48 BD DEQUEVAUVILLIERS,GENNEVILLIERS,92230,France,FR
1,FR43,1000053993,1 RUE DU WAGE,BEAUVAIS,60000,France,FR
2,HU07,1500013142,12,,1612,Bulgaria,BG
3,FR45,1000064589,12 RUE DE BITBOURG,LUXEMBOURG,1273,Luxembourg,LU
4,FR43,1000058890,10 AVENUE DES CARREAUX ZI LES FOU,VERRIERES EN ANJOU,49480,France,FR
...,...,...,...,...,...,...,...
34233,FR45,1500075716,4 ROUTE DE VOUZAILLES,MAISONNEUVE,86170,France,FR
34234,FR43,1000071105,2 RUE ROUGET DE LISLE,ISSY LES MOULINEAUX,92130,France,FR
34235,FR45,1000065981,45A RUE LYAUTEY,GENEUILLE,25870,France,FR
34236,HU07,CAT01010,GEWERBEPARK OST 312,HARTBERG,8230,Austria,AT


#### Geopy and Bing
* https://towardsdatascience.com/driving-distance-between-two-or-more-places-in-python-89779d691def
* https://geopy.readthedocs.io/en/stable/#
* unlimited geolocation

In [18]:
def get_longitude_latitude(x, prefix):
    '''
    Getting longitude and latitude
    prefix_lat_long is either in this case either
    Shipper or Receiver
    '''
    BING_API_KEY = os.getenv('BING_API_KEY')
    geolocator = Bing(
        api_key=BING_API_KEY)
    # Rue Roland Garros, 27930 Guichainville, France
    address = str(x[f'{prefix} address'])+','+str(x[f'{prefix} zip']) + \
        ' '+str(x[f'{prefix} city'])+','+str(x[f'{prefix} country code'])
    location = geolocator.geocode(address)
    if location is not None:
        return location.latitude, location.longitude
    else:
        address = str(x[f'{prefix} zip'])+' '+str(x[f'{prefix} city']
                                                  )+','+str(x[f'{prefix} country code'])
        location = geolocator.geocode(address)
        if location is not None:
            return location.latitude, location.longitude
        else:
            address = str(x[f'{prefix} city'])+',' + \
                str(x[f'{prefix} country code'])
            location = geolocator.geocode(address)
            if location is not None:
                return location.latitude, location.longitude
            else:
                return NaN, NaN


In [23]:
# Get geolocation
BING_API_KEY = os.getenv('BING_API_KEY')
geolocator = Bing(api_key=BING_API_KEY)
address = "Rue Roland Garros, 27930 Guichainville, France"
location = geolocator.geocode(address)

In [24]:
location

Location(Rue Roland Garros, Guichainville, Normandy 27930, France, (49.00580558, 1.17943339, 0.0))

https://www.natasshaselvaraj.com/a-step-by-step-guide-on-geocoding-in-python/ 

## Get distance

### AWS

In [33]:
# Import one AWS json file 
s3 = s3fs.S3FileSystem()
path = s3.ls(f'{project_path}/processed/aws')[1]
with s3.open(f's3://{path}', 'r') as fp:
        aws_json = json.load(fp)
line_string = aws_json['Geometry']['LineString']

print('Distance with AWS:', aws_json['Distance'])

Distance with AWS: 367.065


### Geopy

In [35]:
'''
Distance of the line_string using geopy.distance.geodesic
'''

import geopy.distance
sum = 0
for i in range(1,len(line_string)):
    x = geopy.distance.geodesic(line_string[i-1], line_string[i]).km
    sum = sum+x
print('Distance with Geopy: ', sum)

Distance with Geopy:  416.49912979604375


In [38]:
'''
Distance with Harversine forumla
'''

from math import sin, cos, sqrt, atan2, radians
def get_distance(point1, point2):
    R = 6370
    lat1 = radians(point1[0])  #insert value
    lon1 = radians(point1[1])
    lat2 = radians(point2[0])
    lon2 = radians(point2[1])

    dlon = lon2 - lon1
    dlat = lat2- lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    distance = R * c
    return distance

sum = 0
for i in range(1,len(line_string)):
    x = get_distance(line_string[i-1], line_string[i])
    sum = sum+x
print('Distance with harversine formula: ', sum)

Distance with harversine formula:  417.22230459661506


### Open Street Map Routing (OSRM)

In [40]:
'''
Distance with OSRM
'''

import requests
def get_distance(a, b):
    '''
    Getting distance with 
    '''

    # call the OSMR API
    r = requests.get(
        f"http://router.project-osrm.org/route/v1/car/{a[0]},{a[1]};{b[0]},{b[1]}?overview=false""")
    # then you load the response using the json libray
    # by default you get only one alternative so you access 0-th element of the `routes`
    routes = json.loads(r.content)
    try:
        route_1 = routes.get("routes")[0]
        return route_1['distance']
    except:
        return None

print('Distance with OSRM:', get_distance(line_string[0], line_string[-1])/1000)

Distance with OSRM: 371.1101


> OSRM and AWS quite similar results

* also possible to get waypoints with OSRM

In [45]:
from src.distance import get_waypoints_osrm
waypoints = get_waypoints_osrm(line_string[0], line_string[-1])['geometry']['coordinates']

### Shapely

In [47]:
from shapely.geometry import LineString
from shapely.ops import transform
from functools import partial
import pyproj

line1 = LineString(line_string)
print(str(line1.length) + " degrees")
# 0.0115488362184 degrees

# Geometry transform function based on pyproj.transform
project = partial(
    pyproj.transform,
    pyproj.Proj('EPSG:4326'),
    pyproj.Proj('EPSG:32633'))

line2 = transform(project, line1)
print(str(line2.length) + " meters")

3.752793983101279 degrees
478088.42914114764 meters


c:\Users\SESA667490\venvs\sustainable-transport\lib\site-packages\shapely\ops.py:274: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  return type(geom)(zip(*func(*zip(*geom.coords))))


### Openrouteservice

* Limited to 2000 per day and 40 per minute
* On premise geolocation not available

In [ ]:
OPENROUTESERVICE_API_KEY = os.getenv('OPENROUTESERVICE_API_KEY')

In [ ]:
coords = ((8.34234, 48.23424), (8.34423, 48.26424))
# Specify your personal API key
client = openrouteservice.Client(key=OPENROUTESERVICE_API_KEY)
routes = client.directions(coords)

In [ ]:
geometry = client.directions(coords)['routes'][0]['geometry']
decoded = openrouteservice.convert.decode_polyline(geometry)

In [ ]:
OPENROUTESERVICE_API_KEY = os.getenv('OPENROUTESERVICE_API_KEY')
# Specify your personal API key
client = openrouteservice.Client(key=OPENROUTESERVICE_API_KEY)

# set location coordinates in longitude,latitude order
shipment = data_test.iloc[1]

coords = (
    (shipment['Shipper longitude'], shipment['Shipper latitude']),
    (shipment['Receiver longitude'], shipment['Receiver latitude'])
)
routes = client.directions(coords)
geometry = routes['routes'][0]['geometry']

decoded = openrouteservice.convert.decode_polyline(geometry)


### Distances per country

In [ ]:
from shapely.geometry import Point, Polygon, LineString

# Create Point objects
p1 = Point(24.952242, 60.1696017)
p2 = Point(10.976567, 60.1612500)
line = LineString([[24.952242, 60.1696017],[10.976567, 60.1612500]])
europe[line.intersects(europe.geometry)]
europe[europe['name']=='Norway'][['geometry']].values[0]

### Others codes for geolocation

#### OpenStreetMap

In [ ]:
def get_distance(x):
    # call the OSMR API
    r = requests.get(
        f"http://router.project-osrm.org/route/v1/car/{x['Shipper longitude']},{x['Shipper latitude']};{x['Receiver longitude']},{x['Receiver latitude']}?overview=false""")
    # then you load the response using the json libray
    # by default you get only one alternative so you access 0-th element of the `routes`
    routes = json.loads(r.content)
    try:
        route_1 = routes.get("routes")[0]
        return route_1['distance']
    except:
        return None


In [ ]:
data_test['distance'] = data_test.apply(lambda x: get_distance(x), axis=1)

Other ways to use OpenStreetMap:
* https://medium.com/@nilufarmohammadi1/find-the-best-route-with-openstreetmap-using-python-da70eff5b1ac


In [ ]:
x = data_test.iloc[0]
# call the OSMR API
r = requests.get(
    f"http://router.project-osrm.org/route/v1/car/{x['Shipper longitude']},{x['Shipper latitude']};{x['Receiver longitude']},{x['Receiver latitude']}?overview=false""")
# then you load the response using the json libray
# by default you get only one alternative so you access 0-th element of the `routes`
routes = json.loads(r.content)
try:
    route_1 = routes.get("routes")[0]
except:
    pass
